In [286]:
%run Utils.ipynb
%run CollaborativeFiltering.ipynb

In [290]:
class Recommender():
    
    def __init__(self, batchSize = None , numFeature = 10, maxItr = 1000, lam = 10, ):
        
        self.batchSize = batchSize
        self.numFeature = numFeature
        self.maxItr = maxItr
        self.lam = lam
        
    def fit(self, Y, R, YMean=None, X = None, Theta = None, learner = None, fileName = None, animate=False):
        
        YNorm, YMean = Normalize.normalize(Y, R, YMean)
        numMovie, numUser = Y.shape

        if learner == 'user':
            if X is None:
                raise ValueError('Learner is user, X (Movie Gradient) can not be None')
                
            Theta = rand.rand(numUser, self.numFeature)
        elif learner == 'movie':
            if Theta is None:
                raise ValueError('Learner is movie, Theat (User Gradient) can not be None')
                
            X = rand.rand(numMovie, self.numFeature)
        elif learner is None:
            if X is None:
                X = rand.rand(numMovie, self.numFeature)
            
            if Theta is None:
                Theta = rand.rand(numUser, self.numFeature)
        else:
            raise ValueError('learner cane either be user/movie/None, but is {}'.format(learner))
            
        initialParam = np.concatenate((X, Theta)).ravel()

        anim = AnimateScores(maxItr) if animate else None
        # Some variables used for plotting
        costs = []
        counts= []
        #itr = []
        
        # This is default implementation of Scipy, however I have doubts about it. :)
        learning_rate = [(1. / (1. + 0.01 * it)) for it in  range(self.maxItr)]

        options = {'eps': learning_rate, 'maxiter': self.maxItr, 'disp': True}
        
        ### For implemntation of static COFI.Below is the example
        #args = (YNorm, R, numUser, numMovie, numFeature, lam, batchSize, learner ,anim ,costs, counts)
        #optimum = minimize(CollaborativeFilter.cost, initialParam, args=args, jac=CollaborativeFilter.gradient, \
        #        method='CG',options=options)
        
        cofi = CollaborativeFilter(YNorm, R, numUser, numMovie, self.numFeature, self.lam, self.batchSize, learner ,anim ,costs, counts)
        
        if self.batchSize is None:
            print('...Gradient descent...')
            fileName = fileName + '-GD'
            optimum = minimize(cofi.cost, initialParam, jac=cofi.gradient, method='CG',options=options)
        else:
            print('...Stochastic gradient descent...')
            fileName = fileName + '-SGD'
            optimum = minimize(cofi.cost, initialParam, jac=cofi.stochasticGradient, method='CG',options=options)

        # Saving the graphs
        if fileName is not None:
            print('Saving images...')
            AnimateScores.plot_static_graph(costs, file = fileName)
            if anim is not None:
                anim.save(fileName)
        
        print('Optimal cost is {} with status {} after {} iterations '.format(optimum.fun, optimum.success, optimum.nit))
        
        theta = optimum.x
        X = np.reshape(theta[0:numMovie * self.numFeature], (numMovie, self.numFeature))
        Theta = np.reshape(theta[numMovie * self.numFeature:], (numUser, self.numFeature))
        
        return X, Theta, YMean
        
    def predict(self, X, Theta, YMean):
        return X.dot(Theta.T) + YMean
        
    def giveRatings(self, movieFileName):
        
        self.movieList = DATA.readTXT(movieFileName)
        myMovieRating = np.zeros((len(self.movieList), 1))

        myMovieRating[0] = 4
        myMovieRating[97] = 2
        myMovieRating[6] = 3
        myMovieRating[11] = 5
        myMovieRating[53] = 4
        myMovieRating[63] = 5
        myMovieRating[65] = 3
        myMovieRating[68] = 5
        myMovieRating[182] = 4
        myMovieRating[225] = 5
        myMovieRating[354] = 5

        for i in range(0, len(myMovieRating)):
            if myMovieRating[i] != 0:
                print('Rated {} for movie {}'.format(int(myMovieRating[i]), self.movieList[i]))
        
        return myMovieRating
        
    def recommend(self, prediction):
        
        myRecommendations = np.reshape(prediction[:, 0], (len(prediction[:,0]), 1))
        sortedIndex = np.argsort(myRecommendations, axis=0)[::-1]
        
        print('\nTop Recommendations:- ')
        for i in range(0, 10):
            j = sortedIndex[i].item()
            print(' Rating {} for movie {}'.format(int(myRecommendations[j]), self.movieList[j]))
            
        return myRecommendations

    def saveTrainedData(self, file, X, Theta):
        trainedData = {'X': X, 'Theta': Theta}
        spi.savemat(file, trainedData)
        
    def rms(self, y, y_hat):

        rms = mean_squared_error(y, y_hat)
        print("Root mean squared error : {}".format(rms))
        

In [291]:
def RecommenderModel(dataFile, movieFileName, maxItr = 1000,  batch_size=None, fileName=None):
    
    Y, R = DATA.loadData(file=dataFile)
    
    model = Recommender(maxItr = maxItr, batchSize=batch_size)
    
    print('----Given Raitings by users------\n')
    y = model.giveRatings(movieFileName)
    
    Y = np.concatenate((y, Y), axis=1)
    R = np.concatenate(((y > 0).astype(int), R), axis=1)
    
    print('\n----Training------\n')
    X, Theta, YMean = model.fit(Y, R, fileName = fileName)
    
    y_hat = model.predict(X, Theta, YMean)

    print('\n----Recommendations------')
    y_recommended = model.recommend(y_hat)
    
    print('\n----RMS error---')
    model.rms(y, y_recommended)

In [296]:
def onlineRecommenderModel(dataFile, movieFileName, maxItr = 1000,  batch_size=None, fileName=None):
    
    Y, R = DATA.loadData(file=dataFile)
    
    model = Recommender(maxItr = maxItr, batchSize=batch_size)
    
    print('\n----Training------\n')
    X, Theta, YMean = model.fit(Y, R, fileName = "{}-fit".format(fileName))
    print(X.shape, Theta.shape)
    print('\n----Given Raitings by users------\n')
    y = model.giveRatings(movieFileName)
    r = np.where(y > 0, 1, 0).astype(int)
    
    print('\n----Training------')
    x, theta, yMean = model.fit(y, r,YMean = YMean, X = X, Theta=None, learner='user', \
                                fileName = "{}-OL-fit".format(fileName))
    print(x.shape, theta.shape)
    y_hat = model.predict(x, theta, yMean)

    print('\n----Recommendations------')
    y_recommended = model.recommend(y_hat)
    
    print('\n----RMS error---')
    model.rms(y, y_recommended)